In [1]:
import utilities
import matplotlib.pyplot as plt
import numpy as np
import AdaBoost
import KernelPerceptron
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


In [2]:
height = width = 16
class_number = 10
N = 2

#train_type = 'one_Vs_all'
train_type = 'one_Vs_one'

#algorithm = 'AdaBoost'
algorithm = 'KernelPerceptron'
#algorithm = 'SVM'

kernel_type = 'gaussian'
#kernel_type = 'polynomial'
#kernel_type = ' '

#parameters = [1,2,3,4,5,6,7]
#parameters = [0.01,0.1,1]
parameters = [0.1]
#parameters = [30,50,60]
#parameters = [0.5,1,1.5]

EXPERIMENT_TYPE = 1

In [3]:
train_data = 'Datasets/full_dataset.dat'
data,labels = utilities.read_data(train_data,width,height)
print(data.shape)
print(labels.shape)

(9297, 256)
(9297,)


In [4]:
if(EXPERIMENT_TYPE == 1):
    
    ## N runs for random train test configurations
    results = np.zeros((N,len(parameters)))
    
    confusion_error_list = []

    for run_index in range(0,N):
    
        train_data,test_data,train_labels,test_labels = utilities.data_split(data,labels,0.2)
    
        data_per_class_dictionary = utilities.collect_each_class_images(train_data,train_labels,class_number)
        pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
        #print("Paired classifiers number: " + str(len(pair_datasets)))

        #print(train_data.shape)
        #print(test_data.shape)
        #print(train_labels.shape)
        #print(test_labels.shape)
        for i in range(0,len(parameters)):
        
            param = parameters[i]
        
            param_list = []
            param_list = param_list + [kernel_type]
            param_list  = param_list + [float(param)]
            print(train_type + " training for algorithm " + algorithm + " with params " + str(param_list) +  " and random split " + str(run_index+1))
        
            models_dict = None
            pair_models = None
            if(train_type == 'one_Vs_all'):
                models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,param_list)
            else:
                pair_models = utilities.one_vs_one_training(pair_datasets,algorithm,param_list)
         
            test_predictions = None
            if(train_type == 'one_Vs_all'):
                test_predictions,test_confidence,average_confidence = utilities.one_VS_all_testing(algorithm,models_dict,test_data,test_labels)
            else:
                test_predictions = utilities.one_vs_one_testing(algorithm,pair_models,test_data,class_number)
           
            results[run_index][i] = utilities.calculate_accuracy(test_predictions,test_labels)
            
            confusion_error = utilities.confusion_matrix(test_predictions,test_labels)
            
            confusion_error_list.append(confusion_error)
            
            
    for i in range(results.T.shape[0]):
        print("Mean error and std for parameter: " + str(parameters[i]) + " is " + str(np.mean(results.T[i])) + " - " + str(np.std(results.T[i])))
        
    utilities.get_average_confusions(confusion_error_list)

one_Vs_one training for algorithm KernelPerceptron with params ['gaussian', 0.1] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['gaussian', 0.1] and random split 2
Mean error and std for parameter: 0.1 is 96.07526881720429 - 0.16129032258064768
(0, 0) (0.0, 0.0)
(0, 1) (0.0, 0.0)
(0, 2) (2.5, 0.5)
(0, 3) (0.5, 0.5)
(0, 4) (0.0, 0.0)
(0, 5) (2.0, 1.0)
(0, 6) (3.0, 1.0)
(0, 7) (0.0, 0.0)
(0, 8) (2.5, 0.5)
(0, 9) (0.5, 0.5)
(1, 0) (0.0, 0.0)
(1, 1) (0.0, 0.0)
(1, 2) (0.0, 0.0)
(1, 3) (0.0, 0.0)
(1, 4) (0.5, 0.5)
(1, 5) (0.0, 0.0)
(1, 6) (0.0, 0.0)
(1, 7) (0.0, 0.0)
(1, 8) (0.0, 0.0)
(1, 9) (0.0, 0.0)
(2, 0) (2.0, 2.0)
(2, 1) (0.0, 0.0)
(2, 2) (0.0, 0.0)
(2, 3) (1.0, 0.0)
(2, 4) (1.0, 1.0)
(2, 5) (0.5, 0.5)
(2, 6) (0.5, 0.5)
(2, 7) (0.0, 0.0)
(2, 8) (0.0, 0.0)
(2, 9) (0.0, 0.0)
(3, 0) (1.5, 0.5)
(3, 1) (0.0, 0.0)
(3, 2) (1.0, 1.0)
(3, 3) (0.0, 0.0)
(3, 4) (0.0, 0.0)
(3, 5) (2.0, 0.0)
(3, 6) (0.0, 0.0)
(3, 7) (0.0, 0.0)
(3, 8) (4.0, 0.0)
(3, 9) (0.0, 0.0)

In [5]:
#hard_items_indexes_1,hard_items_indexes_2 = utilities.get_hardest_to_predict_items(test_confidence,average_confidence,test_predictions,test_labels)
#print(hard_items_indexes_1)
#print(hard_items_indexes_2)
#hard_items_indexes = hard_items_indexes_2
#for index in hard_items_indexes:
#    imgplot = plt.imshow(test_data[index].reshape((16,16)))
#    plt.show()